In [89]:
#Importando Bibliotecas
import pandas as pd
import numpy as np
import os
import datetime
from datetime import timedelta
import math
from pathlib import Path

In [2]:
#Selecionando as Variáveis
ano_dt_base = 2020
tx_juros = 0.06
cres_sal = 0.01
cres_benef = 0.01

Colunas que vai precisar pro cálculo:

LOTAÇÃO	MATRICULA	GRUPO	CRITERIO DE APOSENTADORIA	SEXO	DTNASC	DTADM	DATACARGO	SALPART	CARREIRA	TSANT	CONJUGE	DTNASCONJ	DTNASDEPINV	DTNASDEPVAL

In [77]:
#Importando as bases
bd_ativos = pd.read_csv(str(Path(os.getcwd())) + '\\1.SERVIDORES.csv', encoding='UTF-8', sep=';')#, thousands=',')

#Convertendo os dados para float
bd_ativos['DT_NASC_SERVIDOR'] = pd.to_datetime(bd_ativos['DT_NASC_SERVIDOR'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_ING_ENTE'] = pd.to_datetime(bd_ativos['DT_ING_ENTE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_ING_CARGO'] = pd.to_datetime(bd_ativos['DT_ING_ENTE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['VL_BASE_CALCULO'] = bd_ativos['VL_BASE_CALCULO'].str.replace(',', '.').astype(float)
bd_ativos['DT_NASC_CONJUGE'] = pd.to_datetime(bd_ativos['DT_NASC_CONJUGE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_NASC_NOVO'] = pd.to_datetime(bd_ativos['DT_NASC_NOVO'], yearfirst=True, format='%d/%m/%Y')

In [119]:
bd_ativos.DT_ING_ENTE

0     2000-06-30
1     2017-11-01
2     1999-04-02
3     2013-01-14
4     2013-04-08
         ...    
357   1998-02-02
358   2019-03-20
359   2009-09-28
360   2007-03-06
361   2007-02-07
Name: DT_ING_ENTE, Length: 362, dtype: datetime64[ns]

In [120]:
#Calculando as Estatísticas dos Ativos
statis_ativos = pd.DataFrame()

statis_ativos['IDADE_SERVIDOR'] = ano_dt_base - bd_ativos.DT_NASC_SERVIDOR.dt.strftime('%Y').astype(int) #Ano da Data-Base - Data de Nascimento do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_ADM_ARRED'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365)).astype(int) #Data de Ingresso no Ente - Data de Nascimento do Servidor
statis_ativos['IDADE_ADM_SEM_ARRED'] = (bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365)
statis_ativos['IDADE_ADM_ARRED_CIMA'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365) + 0.5).astype(int)
statis_ativos['IDADE_ADM_ARRED_BAIXO'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365) - 0.5).astype(int) 
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_CONJUGE'] = bd_ativos.DT_NASC_CONJUGE.mask(bd_ativos.DT_NASC_CONJUGE.isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos.IDADE_CONJUGE = ano_dt_base - statis_ativos.IDADE_CONJUGE.dt.strftime('%Y').astype(int) #Calculando as Idades dos Cônjuges | Ano da Data-Base - Data de Nascimento do Cônjuge
statis_ativos.loc[(bd_ativos.CO_EST_CIVIL_SERVIDOR == 1) | (bd_ativos.CO_EST_CIVIL_SERVIDOR == 3) | (bd_ativos.CO_EST_CIVIL_SERVIDOR == 4) | (bd_ativos.CO_EST_CIVIL_SERVIDOR == 5), 'IDADE_CONJUGE'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_NOVO'] = bd_ativos.DT_NASC_NOVO.mask(bd_ativos.DT_NASC_NOVO.isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos.IDADE_NOVO = ano_dt_base - statis_ativos.IDADE_NOVO.dt.strftime('%Y').astype(int) #Calculando as Idades do Filho Mais Novo | Ano da Data-Base - Data de Nascimento do Filho Mais Novo
statis_ativos.loc[statis_ativos.IDADE_NOVO > 21, 'IDADE_NOVO'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['TP_PREFEITURA'] = round((2020-12-31 - bd_ativos.DT_ING_ENTE) / timedelta(days=365) - 0.5).astype(int) 
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['DIF_ETARIA'] = statis_ativos.IDADE_CONJUGE - statis_ativos.IDADE_SERVIDOR #Calculando a Diferença Etária do Cônjuge

#statistics_ativos['TSANT'] = bd_ativos['DT_ING_ENTE'] - pd.to_timedelta(bd_ativos['NU_TEMPO_RGPS'], unit='d')
statis_ativos.head(15)

TypeError: Addition/subtraction of integers and integer-arrays with DatetimeArray is no longer supported.  Instead of adding/subtracting `n`, use `n * obj.freq`

In [73]:
#Calculando as Estatísticas dos Ativos
statis_ativos = pd.DataFrame()

statis_ativos['IDADE_SERVIDOR'] = ano_dt_base - bd_ativos['DT_NASC_SERVIDOR'].dt.strftime('%Y').astype(int) #Ano da Data-Base - Data de Nascimento do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
#statis_ativos['IDADE_ADM'] =  bd_ativos['DT_ING_ENTE'].dt.strftime('%Y').astype(int) - bd_ativos['DT_NASC_SERVIDOR'].dt.strftime('%Y').astype(int) #Data de Ingresso no Ente - Data de Nascimento do Servidor
statis_ativos['IDADE_ADM'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365)).astype(int) #Data de Ingresso no Ente - Data de Nascimento do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_CONJUGE'] = bd_ativos['DT_NASC_CONJUGE'].mask(bd_ativos['DT_NASC_CONJUGE'].isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos['IDADE_CONJUGE'] = ano_dt_base - statis_ativos['IDADE_CONJUGE'].dt.strftime('%Y').astype(int) #Calculando as Idades dos Cônjuges | Ano da Data-Base - Data de Nascimento do Cônjuge
statis_ativos.loc[(bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 1) | (bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 3) | (bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 4) | (bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 5), 'IDADE_CONJUGE'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_NOVO'] = bd_ativos['DT_NASC_NOVO'].mask(bd_ativos['DT_NASC_NOVO'].isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos['IDADE_NOVO'] = ano_dt_base - statis_ativos['IDADE_NOVO'].dt.strftime('%Y').astype(int) #Calculando as Idades do Filho Mais Novo | Ano da Data-Base - Data de Nascimento do Filho Mais Novo
statis_ativos.loc[statis_ativos['IDADE_NOVO'] > 21, 'IDADE_NOVO'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['DIF_ETARIA'] = statis_ativos['IDADE_CONJUGE'] - statis_ativos['IDADE_SERVIDOR'] #Calculando a Diferença Etária do Cônjuge

#statistics_ativos['TSANT'] = bd_ativos['DT_ING_ENTE'] - pd.to_timedelta(bd_ativos['NU_TEMPO_RGPS'], unit='d')
statis_ativos.head(15)

11337.0

In [114]:
statis_ativos['IDADE_ADM_ARRED'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365)).astype(int)
statis_ativos['IDADE_ADM_SEM_ARRED'] = (bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365)
statis_ativos['IDADE_ADM_ARRED_CIMA'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365) + 0.5).astype(int)
statis_ativos['IDADE_ADM_ARRED_BAIXO'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365) - 0.5).astype(int)
statis_ativos[['IDADE_ADM_ARRED', 'IDADE_ADM_SEM_ARRED', 'IDADE_ADM_ARRED_CIMA', 'IDADE_ADM_ARRED_BAIXO']]

,IDADE_ADM_ARRED,IDADE_ADM_SEM_ARRED,IDADE_ADM_ARRED_CIMA,IDADE_ADM_ARRED_BAIXO
0,20,20.120548,21,20
1,43,42.501370,43,42
2,27,27.041096,28,27
3,23,23.304110,24,23
4,35,35.161644,36,35
...,...,...,...,...
357,23,22.720548,23,22
358,24,24.361644,25,24
359,25,25.284932,26,25
360,30,30.260274,31,30


In [111]:
statis_ativos.IDADE_ADM3.sum()

11154